# AI Journalist
AI-Powered Article Generator: Collects and analyzes the latest news to craft high-quality articles autonomously.

AI 기반 자동 기사 생성 시스템: 최신 뉴스를 수집하고 분석하여 고품질의 기사를 작성합니다.

# What we want to know
1. 기존 기사 데이터를 바탕으로 새로운 기사를 생성할 수 있는가
2. Cost effective 하면서도 높은 품질의 기사를 생성할 수 있는가

# Pipeline
1. 필요한 라이브러리 설치 및 API 키 설정
2. 주제 설정 ("아이폰 16 출시 및 스펙")
3. AI(Claude)를 사용해 주제와 관련된 5개의 검색어 생성 (Claude 3 Haiku)
4. 생성된 검색어로 Google 뉴스 검색 수행 (SerpAPI)
5. 각 검색 결과에서 AI가 가장 관련성 높은 기사 선택 (Claude 3 Haiku)
6. 선택된 기사들의 본문 내용 추출
7. 추출된 내용을 바탕으로 AI가 한국어로 NYT 스타일의 기사 작성 (Claude 3.5 Sonnet)

# API Usage while development
Total cost: $0.3464 (461.29 South Korean won)

Claude 3 Haiku

Input: $0.0049 (0.019 Million Input Tokens)
Output: $0.0015 (0.001 Million Output Tokens)

Claude 3.5 Sonnet

Input: $0.21 (0.071 Million Input Tokens)
Output: $0.13 (0.008 Million Output Tokens)

# Reference
https://github.com/mshumer/ai-journalist

%pip install anthropic
%pip install google-search-results
%pip install newspaper3k

# API keys
https://console.anthropic.com/settings/keys
https://serpapi.com/dashboard

In [1]:
import os
from dotenv import load_dotenv
import anthropic

load_dotenv()

# Make sure to set your OpenAI API key in your environment variables
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
SERP_API_KEY = os.getenv("SERP_API_KEY")

# Topic

In [2]:
topic = "OpenAI의 GPT o1-preview 발표 및 상세 내용"
print(topic)

OpenAI의 GPT o1-preview 발표 및 상세 내용


# Search Terms
Topic을 바탕으로 5개의 검색어 생성

상대적으로 저렴한 Claude 3 Haiku 모델을 사용하여 검색어 생성

In [3]:

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
system_prompt = "You are a world-class journalist. Gernerate a list of 5 search terms to search for to research and write an article about the topic."
messages = [{
    "role": "user",
    "content": [{
        "type": "text",
        "text": f"Please provide a list of 5 search terms related to '{topic}' for researching and writing an article. Respond with the search terms in Python-parseable list, separated by commas. Ensure the search terms are specific to the topic and not too general. Ensure the search terms are written in English.\n\ne.g. ['search term 1', 'search term 2', 'search term 3', 'search term 4', 'search term 5']",
    }]
}]
message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=200,
    temperature=0.5,
    system=system_prompt,
    messages=messages,
)
print(message.content[0].text)

['OpenAI GPT-3 preview', 'GPT-3 model capabilities', 'GPT-3 technical details', 'GPT-3 potential applications', 'GPT-3 ethical considerations']


# Parse search terms
String을 Python list 형식으로 변환

In [4]:
import re

response = message.content[0].text
list_pattern = r'\[(.*?)\]'
match = re.search(list_pattern, response)

if match:
    list_string = match.group(1)
    search_terms = [term.strip().strip("'") for term in list_string.split(',')]
    print(search_terms)
else:
    print("리스트를 찾을 수 없습니다.")

['OpenAI GPT-3 preview', 'GPT-3 model capabilities', 'GPT-3 technical details', 'GPT-3 potential applications', 'GPT-3 ethical considerations']


# Search for news articles
1. 각 검색어에 대해 뉴스 검색 수행 (Google Search)
2. 검색 결과에서 가장 관련성 높은 기사 선택 (Claude 3 Haiku)

In [5]:
from serpapi import GoogleSearch

relevant_urls = []

for term in search_terms:
    params = {
        "engine": "google",
        "q": term,
        "api_key": SERP_API_KEY,
        "tbs": "sbd:1", # sort by date
        "tbm": "nws", # news
        "location": "New York, New York, United States",
        "gl": "us",
        "hl": "en",
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    news_results = results["news_results"]

    news_results_title = "\n".join([f"{i+1}. {result['title']}" for i, result in enumerate(news_results)])

    system_prompt = "You are a jounalist assistant. From the given search results, select the title that seems most relevant and informative for writing an article on the topic."
    messages = [{
        "role": "user",
        "content": [{
            "type": "text",
            "text": f"Search Results:\n{news_results_title}\n\nPlease select the single number of the title that seems most relevant for writing an article on the topic '{topic}'. Respond with the single number of the title that seems most relevant. Ensure return the number only.\ne.g. 1",
        }]
    }]

    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=2,
        temperature=0,
        system=system_prompt,
        messages=messages,
    )
    relevant_id = int(message.content[0].text) - 1
    print(relevant_id)
    relevant_urls.extend([news_results[relevant_id]["link"]])

print(relevant_urls)

5
0
2
4
7
['https://www.cgwall.com/usa/introducing-openai-o1-a-leap-forward-in-ai-reasoning-with-new-preview-models/', 'https://fortune.com/2024/09/13/openai-o1-strawberry-model-9-things-you-need-know/', 'https://www.techtarget.com/whatis/feature/GPT-4o-explained-Everything-you-need-to-know', 'https://cointelegraph.com/explained/what-is-gpt-4o-and-how-is-it-different-from-gpt-3-gpt-35-and-gpt-4', 'https://www.analyticsinsight.net/chatgpt/evolution-of-chatgpt-from-gpt-3-to-gpt-4-and-beyond']


In [6]:
news_results[0]

{'position': 1,
 'link': 'https://analyticsindiamag.com/topics/best-chatgpt-alternatives/',
 'title': 'The 10 Best ChatGPT Alternatives in 2024',
 'source': 'Analytics India Magazine',
 'date': '3 weeks ago',
 'snippet': "Google Gemini is a strong alternative to ChatGPT 4 due to its advanced \nmultimodal capabilities, integration with Google's ecosystem, and powerful \nreasoning...",
 'thumbnail': 'https://serpapi.com/searches/66e676e258e4339428d03e5d/images/adb569ddcebf2e9e3cbdde7e168d9f5f151339a0e551bbbaeb9aad5f5850b895.jpeg'}

# Extract article content
각 기사의 본문 내용 크롤링 및 추출, 파싱 (Newspaper3k)

In [7]:
from newspaper import Article

article_texts = ""

for url in relevant_urls:
    try:
        article = Article(url)
        article.download()
        article.parse()
        article_texts += article.text
    except Exception as e:
        print(f"Error parsing article from {url}: {e}")

print(len(article_texts))
print(article_texts[:500])

34485
1. Welcome to OpenAI o1

The o1 series is the first in a new line of AI models focused on enhanced reasoning capabilities. This series, starting with the o1-preview and o1-mini, is designed to think more deeply and systematically before delivering responses. The result? Improved performance in tackling intricate problems across a variety of domains, including science, coding, and mathematics.

2. Advanced Reasoning for Complex Tasks

Unlike its predecessors, the o1 models leverage Chain-of-Thoug


# Generate article
Claude 3.5 Sonnet 모델을 사용하여 고품질의 기사 생성

In [8]:
system_prompt = "You are a journalist. Write a high-quality, NYT-worthy article on the given topic based on the provided article texts. The article should be well-structured, inormative, and engaging to Korean readers."
messages = [{
    "role": "user",
    "content": f"Topic: {topic}\n\nArticle Texts:\n{article_texts}\n\nPlease write a high-quality, NYT-worthy article on the topic based on the provided article texts. The article should be well-structured, informative, and engaging. Ensure the length is at least as long as as a NYT cover story -- at a minimum, 15 paragraphs. Ensure the article is written in Korean."
}]

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=3000,
    temperature=0.5,
    system=system_prompt,
    messages=messages,
)

print(message.content[0].text)

OpenAI, 혁신적인 AI 모델 'GPT o1-preview' 공개

인공지능 기술의 새로운 지평을 열다

OpenAI가 기존 대규모 언어 모델들을 뛰어넘는 추론 및 수학 능력을 갖춘 새로운 AI 모델 가족을 발표했다. 지난 목요일, 이 회사는 'o1-preview'와 'o1-mini'라는 두 가지 모델의 '프리뷰' 버전을 일부 유료 사용자들에게 공개했다.

향상된 추론과 수학 능력을 갖춘 AI는 화학자, 물리학자, 엔지니어들이 복잡한 문제의 해답을 도출하는 데 도움을 줄 수 있어 새로운 제품 개발에 기여할 것으로 기대된다. 또한 투자자들의 옵션 거래 전략 계산이나 재무 설계사들의 위험-보상 균형을 고려한 포트폴리오 구성에도 활용될 수 있다.

더 나은 추론, 계획, 문제 해결 능력은 기술 기업들이 복잡한 작업을 수행할 수 있는 AI 에이전트를 구축하려는 노력에도 필수적이다. 이러한 에이전트는 전체 컴퓨터 프로그램 작성, 웹에서 정보 찾기, 스프레드시트로 데이터 가져오기, 데이터 분석 수행, 결과 요약 보고서 작성 등의 작업을 수행할 수 있을 것으로 예상된다.

OpenAI는 o1 모델에 대한 인상적인 벤치마크 결과를 발표했다. 내부적으로 '스트로베리'라는 코드명으로 불렸던 이 모델은 고등학생들을 위한 AIME 수학 경진대회 문제에서 83.3%의 정답률을 보였다. 이는 GPT-4o의 13.4% 정답률과 비교할 때 큰 진전이다. 또 다른 평가에서는 박사 수준의 과학 문제의 78%를 정확히 답변했는데, 이는 GPT-4o의 56.1%와 인간 전문가의 69.7%를 뛰어넘는 성과다.

o1 모델은 또한 이전 모델들에 비해 '환각' - 그럴듯하지만 부정확한 답변을 자신 있게 제공하는 현상 - 가능성이 크게 낮아졌다고 OpenAI는 밝혔다. 안전 가드레일을 우회하도록 유도하는 '탈옥'도 더 어려워졌다.

o1-preview가 널리 사용 가능해진 후 몇 시간 동안 사용자들이 실시한 테스트에서, 이 모델은 GPT-4와 GPT-4o를 포함한 OpenAI의 가장 강력한 이전 모

In [9]:
with open("news.txt", "w", encoding="utf-8") as f:
    f.write(message.content[0].text)